# imports 

In [1]:
import read_write_qrcd as q_reader
from gensim import corpora, models, similarities
from collections import defaultdict
import pprint

# load data 

In [2]:
train_data_file = '../quranqa/datasets/qrcd_v1.1_train.jsonl'
train = q_reader.load_jsonl(train_data_file)

Loaded 710 records from ../quranqa/datasets/qrcd_v1.1_train.jsonl


# IR function

In [3]:
def ir(data_item={}, query_key='question', docs_key='passage', res_key='answers', index_method='lsi', dims=4) -> list:
    # get data items 
    docs = [ver.strip() for ver in data_item.get(docs_key).split('.') if ver.strip()]
    query = data_item.get(query_key)
    res = data_item.get(res_key)

    stoplist = ''.split()
    texts = [
        [word for word in doc.split() if word not in stoplist]
        for doc in docs
    ]

    # build dictionary and corpus 
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]

    # build tfidf & lsi models 
    tfidf = models.TfidfModel(corpus) 
    corpus_tfidf = tfidf[corpus]
    lsi_model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=dims)
    corpus_lsi = lsi_model[corpus_tfidf]

    # generate query BOW vector and LSI vector from the question 
    query_bow = query.replace('؟', '').replace('"', '').split()
    print('query bow:', query_bow)
    vec_bow = dictionary.doc2bow(query_bow)
    vec_lsi = lsi_model[vec_bow]

    result = []
    try:
        # generate tfidf and lsi indices 
        lsi_index = similarities.MatrixSimilarity(corpus_lsi)
        tfidf_index = similarities.MatrixSimilarity(corpus_tfidf)
        
        
        # get similarities between queries and docs 
        tfidf_sims = tfidf_index[vec_bow]
        lsi_sims = lsi_index[vec_lsi]

        # sort documents according to similarties 
        
        if index_method == 'tfidf':
            sims = tfidf_sims
        else:
            sims = lsi_sims
        
        
        sims = sorted(enumerate(sims), key=lambda item: -item[1])
        for doc_position, doc_score in sims:
            result.append( (doc_score, docs[doc_position]) )
    except:
        for doc in docs:
            result.append( (0.0, doc) )
    return result

# some examples 

## Example 1

In [4]:
data = train[0]
print('Query (Question):', data.get('question'))
print('Answers:', data.get('answers'))
print('verses sorted by similarity score:')
ir(data_item=data)

Query (Question): لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ""من يضلل الله فما له من هاد"" كما ورد من قوله تعالى في آية 23 و آية 36 من سورة الزمر؟
Answers: [{'text': 'أولئك الذين اشتروا الضلالة بالهدى', 'start_char': 504}]
verses sorted by similarity score:
query bow: ['لماذا', 'سيُحاسب', 'ويُعذب', 'الضال', 'يوم', 'القيامة', 'ان', 'كان', 'من', 'يضلل', 'الله', 'فما', 'له', 'من', 'هاد', 'كما', 'ورد', 'من', 'قوله', 'تعالى', 'في', 'آية', '23', 'و', 'آية', '36', 'من', 'سورة', 'الزمر']


[(0.93099207, 'يخادعون الله والذين آمنوا وما يخدعون إلا أنفسهم وما يشعرون'),
 (0.81462085, 'ومن الناس من يقول آمنا بالله وباليوم الآخر وما هم بمؤمنين'),
 (0.7821177,
  'أولئك الذين اشتروا الضلالة بالهدى فما ربحت تجارتهم وما كانوا مهتدين'),
 (0.4166727, 'ألا إنهم هم المفسدون ولكن لا يشعرون'),
 (0.3769802, 'في قلوبهم مرض فزادهم الله مرضا ولهم عذاب أليم بما كانوا يكذبون'),
 (0.33878773,
  'وإذا قيل لهم آمنوا كما آمن الناس قالوا أنؤمن كما آمن السفهاء ألا إنهم هم السفهاء ولكن لا يعلمون'),
 (0.3092097,
  'وإذا لقوا الذين آمنوا قالوا آمنا وإذا خلوا إلى شياطينهم قالوا إنا معكم إنما نحن مستهزئون'),
 (0.17794453, 'الله يستهزئ بهم ويمدهم في طغيانهم يعمهون'),
 (0.17013404, 'وإذا قيل لهم لا تفسدوا في الأرض قالوا إنما نحن مصلحون')]

## Example 2

In [5]:
data = train[19]
print('Query (Question):', data.get('question'))
print('Answers:', data.get('answers'))
print('verses sorted by similarity score:')
ir(data_item=data)

Query (Question): إن كان الله قدّر علي أفعالي فلماذا يحاسبني؟
Answers: [{'text': 'كل إنسان ألزمناه طائره في عنقه', 'start_char': 141}, {'text': 'من اهتدى فإنما يهتدي لنفسه ومن ضل فإنما يضل عليها', 'start_char': 252}]
verses sorted by similarity score:
query bow: ['إن', 'كان', 'الله', 'قدّر', 'علي', 'أفعالي', 'فلماذا', 'يحاسبني']


[(0.0,
  'وجعلنا الليل والنهار آيتين فمحونا آية الليل وجعلنا آية النهار مبصرة لتبتغوا فضلا من ربكم ولتعلموا عدد السنين والحساب وكل شيء فصلناه تفصيلا'),
 (0.0,
  'وكل إنسان ألزمناه طائره في عنقه ونخرج له يوم القيامة كتابا يلقاه منشورا'),
 (0.0, 'اقرأ كتابك كفى بنفسك اليوم عليك حسيبا'),
 (0.0,
  'من اهتدى فإنما يهتدي لنفسه ومن ضل فإنما يضل عليها ولا تزر وازرة وزر أخرى وما كنا معذبين حتى نبعث رسولا'),
 (0.0,
  'وإذا أردنا أن نهلك قرية أمرنا مترفيها ففسقوا فيها فحق عليها القول فدمرناها تدميرا'),
 (0.0, 'وكم أهلكنا من القرون من بعد نوح وكفى بربك بذنوب عباده خبيرا بصيرا')]

## Example 3

In [6]:
data = train[40]
print('Query (Question):', data.get('question'))
print('Answers:', data.get('answers'))
print('verses sorted by similarity score:')
ir(data_item=data)

Query (Question): ما هو وصف الحور العين؟
Answers: [{'text': 'خيرات حسان', 'start_char': 600}, {'text': 'حور مقصورات في الخيام', 'start_char': 636}, {'text': 'لم يطمثهن إنس قبلهم ولا جان', 'start_char': 683}]
verses sorted by similarity score:
query bow: ['ما', 'هو', 'وصف', 'الحور', 'العين']


[(0.0, 'ولمن خاف مقام ربه جنتان'),
 (0.0, 'فبأي آلاء ربكما تكذبان'),
 (0.0, 'ذواتا أفنان'),
 (0.0, 'فبأي آلاء ربكما تكذبان'),
 (0.0, 'فيهما عينان تجريان'),
 (0.0, 'فبأي آلاء ربكما تكذبان'),
 (0.0, 'فيهما من كل فاكهة زوجان'),
 (0.0, 'فبأي آلاء ربكما تكذبان'),
 (0.0, 'متكئين على فرش بطائنها من إستبرق وجنى الجنتين دان'),
 (0.0, 'فبأي آلاء ربكما تكذبان'),
 (0.0, 'فيهن قاصرات الطرف لم يطمثهن إنس قبلهم ولا جان'),
 (0.0, 'فبأي آلاء ربكما تكذبان'),
 (0.0, 'كأنهن الياقوت والمرجان'),
 (0.0, 'فبأي آلاء ربكما تكذبان'),
 (0.0, 'هل جزاء الإحسان إلا الإحسان'),
 (0.0, 'فبأي آلاء ربكما تكذبان'),
 (0.0, 'ومن دونهما جنتان'),
 (0.0, 'فبأي آلاء ربكما تكذبان'),
 (0.0, 'مدهامتان'),
 (0.0, 'فبأي آلاء ربكما تكذبان'),
 (0.0, 'فيهما عينان نضاختان'),
 (0.0, 'فبأي آلاء ربكما تكذبان'),
 (0.0, 'فيهما فاكهة ونخل ورمان'),
 (0.0, 'فبأي آلاء ربكما تكذبان'),
 (0.0, 'فيهن خيرات حسان'),
 (0.0, 'فبأي آلاء ربكما تكذبان'),
 (0.0, 'حور مقصورات في الخيام'),
 (0.0, 'فبأي آلاء ربكما تكذبان'),
 (0.0, 'لم يطمثهن إنس قبلهم ولا جان')

## Example 4

In [7]:
data = train[100]
print('Query (Question):', data.get('question'))
print('Answers:', data.get('answers'))
print('verses sorted by similarity score:')
ir(data_item=data)

Query (Question): ما هي انواع الحيوانات التي ذكرت في القرآن؟
Answers: [{'text': 'دابة الأرض', 'start_char': 43}]
verses sorted by similarity score:
query bow: ['ما', 'هي', 'انواع', 'الحيوانات', 'التي', 'ذكرت', 'في', 'القرآن']


C:\Users\mo.saad\Anaconda3\envs\quranqa\lib\site-packages\gensim\models\lsimodel.py:101: RuntimeWarning: invalid value encountered in true_divide
  rel_spectrum = np.abs(1.0 - np.cumsum(s / np.sum(s)))


[(0.0,
  'فلما قضينا عليه الموت ما دلهم على موته إلا دابة الأرض تأكل منسأته فلما خر تبينت الجن أن لو كانوا يعلمون الغيب ما لبثوا في العذاب المهين')]

## Example 5

In [8]:
data = train[150]
print('Query (Question):', data.get('question'))
print('Answers:', data.get('answers'))
print('verses sorted by similarity score:')
ir(data_item=data)

Query (Question): من هم القوم الذين حولهم الله إلى قردة؟
Answers: [{'text': 'الذين اعتدوا منكم في السبت', 'start_char': 165}]
verses sorted by similarity score:
query bow: ['من', 'هم', 'القوم', 'الذين', 'حولهم', 'الله', 'إلى', 'قردة']


[(0.8017837,
  'ثم توليتم من بعد ذلك فلولا فضل الله عليكم ورحمته لكنتم من الخاسرين'),
 (0.59761435,
  'ولقد علمتم الذين اعتدوا منكم في السبت فقلنا لهم كونوا قردة خاسئين'),
 (2.9802322e-08, 'فجعلناها نكالا لما بين يديها وما خلفها وموعظة للمتقين'),
 (0.0,
  'وإذ أخذنا ميثاقكم ورفعنا فوقكم الطور خذوا ما آتيناكم بقوة واذكروا ما فيه لعلكم تتقون')]

In [9]:
data = train[150]
print('Query (Question):', data.get('question'))
print('Answers:', data.get('answers'))
print('verses sorted by similarity score:')
ir(data_item=data, index_method='tfidf')

Query (Question): من هم القوم الذين حولهم الله إلى قردة؟
Answers: [{'text': 'الذين اعتدوا منكم في السبت', 'start_char': 165}]
verses sorted by similarity score:
query bow: ['من', 'هم', 'القوم', 'الذين', 'حولهم', 'الله', 'إلى', 'قردة']


[(0.38729835,
  'ثم توليتم من بعد ذلك فلولا فضل الله عليكم ورحمته لكنتم من الخاسرين'),
 (0.28867513,
  'ولقد علمتم الذين اعتدوا منكم في السبت فقلنا لهم كونوا قردة خاسئين'),
 (0.0,
  'وإذ أخذنا ميثاقكم ورفعنا فوقكم الطور خذوا ما آتيناكم بقوة واذكروا ما فيه لعلكم تتقون'),
 (0.0, 'فجعلناها نكالا لما بين يديها وما خلفها وموعظة للمتقين')]